## 데이터 불러오기

In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
# pip install sentence_transformers

In [4]:
# pip install textblob

In [5]:
import pandas as pd
import pymongo
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sklearn
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

import nltk
from nltk.tokenize import RegexpTokenizer       
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download("wordnet")
nltk.download("omw-1.4") 

client=pymongo.MongoClient("192.168.10.240",27017)
db= client['healthcare_sideproject']
FHP_article = db['PMC_FHP_data']
FHP_article = FHP_article.find()
FHP_article = pd.DataFrame(FHP_article)
FHP_article = FHP_article.loc[:,'title':]
FHP_article

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,title,artical_date,abstract_list
0,Influence of forward head posture on muscle ac...,Published online 2022 Nov 14.,Go to:\nAbstract\nForward head posture (FHP) i...
1,Relative posture between head and finger deter...,Published online 2020 Mar 26.,Go to:\nAbstract\nThe hand explores the enviro...
2,Head posture impacts mammalian hyoid position ...,"Published online October 16, 2023.",Go to:\nABSTRACT\nInstantaneous head posture (...
3,Does head and cervical posture correlate to ma...,Published online 2022 Oct 25.,Go to:\nAbstract\nBackground\nThe association ...
4,Changes in deep neck muscle length from the ne...,Published online 2022 Jan 25.,Go to:\nAbstract\nForward head posture (FHP) i...
...,...,...,...
2878,Characteristics of the postural stability of t...,Published online 2023 Jan 4.,Go to:\nAbstract\nObjective: To explore the ch...
2879,Validating an infrared thermal switch as a nov...,Published online 2010 Aug 5.,"Go to:\nAbstract\nBackground\nRecently, a nove..."
2880,How Not to Be Turned into a Zombie,Published online 2018 Oct 31.,Go to:\nAbstract\nThe emerald jewel wasp (Ampu...
2881,Chronic Cervicogenic Tinnitus Rapidly Resolved...,Published online 2016 Mar 23.,Go to:\nAbstract\nIntroduction\nCervicogenic t...


## 데이터 전처리

### 결측치 처리 
- abstract가 없는 값은 제목으로 대체

In [6]:
FHP_article.query("abstract_list==''").count()

title            219
artical_date     219
abstract_list    219
dtype: int64

In [7]:
FHP_article.loc[FHP_article.query("abstract_list==''").index,'abstract_list'] = FHP_article.loc[FHP_article.query("abstract_list==''").index,'title']

In [8]:
FHP_article.query("abstract_list==''")

,title,artical_date,abstract_list


### 중복값 제거

In [9]:
FHP_article.duplicated(subset='title').sum()

3

In [10]:
FHP_article = FHP_article.drop_duplicates(subset='title', keep='first', inplace=False, ignore_index=False)

### 발행일 변환

In [63]:
def date_format(raw):
    raw = raw.replace("Published online","")
    return raw

In [12]:
FHP_article.loc[:,'artical_date']=FHP_article['artical_date'].apply(date_format)


In [13]:
FHP_article

,title,artical_date,abstract_list
0,Influence of forward head posture on muscle ac...,2022 Nov 14.,Go to:\nAbstract\nForward head posture (FHP) i...
1,Relative posture between head and finger deter...,2020 Mar 26.,Go to:\nAbstract\nThe hand explores the enviro...
2,Head posture impacts mammalian hyoid position ...,"October 16, 2023.",Go to:\nABSTRACT\nInstantaneous head posture (...
3,Does head and cervical posture correlate to ma...,2022 Oct 25.,Go to:\nAbstract\nBackground\nThe association ...
4,Changes in deep neck muscle length from the ne...,2022 Jan 25.,Go to:\nAbstract\nForward head posture (FHP) i...
...,...,...,...
2878,Characteristics of the postural stability of t...,2023 Jan 4.,Go to:\nAbstract\nObjective: To explore the ch...
2879,Validating an infrared thermal switch as a nov...,2010 Aug 5.,"Go to:\nAbstract\nBackground\nRecently, a nove..."
2880,How Not to Be Turned into a Zombie,2018 Oct 31.,Go to:\nAbstract\nThe emerald jewel wasp (Ampu...
2881,Chronic Cervicogenic Tinnitus Rapidly Resolved...,2016 Mar 23.,Go to:\nAbstract\nIntroduction\nCervicogenic t...


### 정규화 : 대소문자 통합

In [14]:
def word_lower(raw):
    abstract_list = raw.split()
    modified_list = []
    for abstract_word in abstract_list :
         modified_list.append(abstract_word.lower())
    return " ".join(modified_list)

In [15]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract_list'].apply(word_lower)

/tmp/ipykernel_12512/2255077103.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FHP_article.loc[:,'abstract_clean']=FHP_article['abstract_list'].apply(word_lower)


In [16]:
FHP_article.loc[2,'abstract_clean']

'go to: abstract instantaneous head posture (ihp) can extensively alter resting hyoid position in humans, yet postural effects on resting hyoid position remain poorly documented among mammals in general. clarifying this relationship is essential for evaluating interspecific variation in hyoid posture across evolution, and understanding its implications for hyolingual soft tissue function and swallowing motor control. using didelphis virginiana as a model, we conducted static manipulation experiments to show that head flexion shifts hyoid position rostrally relative to the cranium across different gapes. ihp-induced shifts in hyoid position along the anteroposterior axis are comparable to in vivo hyoid protraction distance during swallowing. ihp also has opposite effects on passive genio- and stylohyoid muscle lengths. high-speed biplanar videoradiography suggests didelphis consistently swallows at neutral to flexed posture, with stereotyped hyoid kinematics across different head postur

### 특수문자, 숫자, '/n', '소제목' 제거

In [17]:
import re
def word_cleaning(raw):
    raw = raw.replace("\n"," ")
    raw = raw.replace("go to: abstract","")
    raw = raw.replace("background:","")
    raw = raw.replace("purpose:","")
    raw = raw.replace("objectives:","")
    raw = raw.replace("objective:","")
    raw = raw.replace("design:","")
    raw = raw.replace("methods:","")
    raw = raw.replace("method:","")
    raw = raw.replace("results:","")
    raw = raw.replace("result:","")
    raw = raw.replace("discussion:","")
    raw = raw.replace("conclusions:","")
    raw = raw.replace("interpretation:","")
    raw = raw.replace("conclusion:","")
    raw = raw.replace("significance:","")
    raw = raw.replace("keywords:","")
    raw = raw.replace("fhp"," ")
    raw = raw.replace("forward head posture"," ")
    raw = raw.replace("head forward posture"," ")
    patterns = '([^a-z|A-Z|\'| ])'
    text_regex = re.sub(pattern = patterns, repl = "",string=raw)
    return text_regex

In [18]:
FHP_article.loc[:,'abstract_clean']=FHP_article['abstract_clean'].apply(word_cleaning)

In [19]:
FHP_article.loc[1789,'abstract_clean']

' background people with low back pain lbp exhibit impaired anticipatory postural adjustment apas objective to evaluate whether current motor retraining treatments address lbpassociated changes in movement coordination during tasks that do and do not require apas design prospectively registered randomized controlled trial with a blinded assessor setting outcome evaluations occurred in a university laboratory treatments in outpatient physical therapy clinics patients fifteen subjects without lbp and  subjects with chronic recurrent nonspecific lbp intervention twelve subjects with lbp received stabilization treatment  received movement system impairment msibased treatment over  weekly hour sessions plus home exercises measurements pre and post treatment surface electromyography emg was recorded bilaterally from trunk and leg muscles during unsupported and supported leglifting tasks which did and did not require an apa respectively vertical reaction forces under the contralateral leg wer

### 단어 토큰화

In [64]:
def word_tokenizing(raw):
    tokenizer = RegexpTokenizer("[\s]+", gaps=True) 
    tokens = tokenizer.tokenize(raw)
    tags = nltk.pos_tag(tokens)
    # nouns_list = [t for t in tags if t[1] in  ['NN','NNP','NNS']]
    nouns_list = [t for t in tags if t[1] in  ['JJ','JJR','NN','NNP','NNS','VB','VBD','VBG','VBN','VBP','VBZ']]
    # nouns_list = [t for t in tags if t[1] in  ['NN','NNP','NNS','VB','VBD','VBG','VBN','VBP','VBZ']]
    return nouns_list

<>:2: DeprecationWarning: invalid escape sequence '\s'
<>:2: DeprecationWarning: invalid escape sequence '\s'
/tmp/ipykernel_12512/2888958303.py:2: DeprecationWarning: invalid escape sequence '\s'
  tokenizer = RegexpTokenizer("[\s]+", gaps=True)


In [21]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_clean'].apply(word_tokenizing)

/tmp/ipykernel_12512/84472225.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FHP_article.loc[:,'abstract_token']=FHP_article['abstract_clean'].apply(word_tokenizing)


In [22]:
FHP_article['abstract_token']

0       [(is, VBZ), (serious, JJ), (problem, NN), (cau...
1       [(hand, NN), (explores, VBZ), (environment, NN...
2       [(instantaneous, JJ), (head, NN), (posture, NN...
3       [(association, NN), (head, NN), (cervical, JJ)...
4       [(is, VBZ), (common, JJ), (postural, JJ), (dev...
                              ...                        
2878    [(explore, VB), (characteristics, NNS), (lower...
2879    [(background, NN), (novel, JJ), (singleswitch,...
2880    [(emerald, NN), (jewel, NN), (wasp, NN), (ampu...
2881    [(introduction, NN), (cervicogenic, NN), (tinn...
2882    [(high, JJ), (visual, JJ), (acuity, NN), (allo...
Name: abstract_token, Length: 2880, dtype: object

### 표제어 추출

In [65]:
def Lemmatization(raw):
    lemmatizer = WordNetLemmatizer()
    lemmatized_list = []
    for word in raw:
        if word[1][0].lower() in ['v','n']:
            lemmatized_word = lemmatizer.lemmatize(word[0],word[1][0].lower())
            if lemmatized_word != 'be':
                lemmatized_list.append(lemmatized_word)
        else:
            lemmatized_list.append(word[0])
    return lemmatized_list

In [24]:
FHP_article.loc[:,'abstract_token']=FHP_article['abstract_token'].apply(Lemmatization)

In [25]:
FHP_article.loc[:,'abstract_token']

0       [serious, problem, cause, head, neck, disabili...
1       [hand, explore, environment, obtain, tactile, ...
2       [instantaneous, head, posture, ihp, alter, res...
3       [association, head, cervical, posture, maloccl...
4       [common, postural, deviation, deep, neck, musc...
                              ...                        
2878    [explore, characteristic, lower, limb, postura...
2879    [background, novel, singleswitch, access, tech...
2880    [emerald, jewel, wasp, ampulex, compressa, ren...
2881    [introduction, cervicogenic, tinnitus, accept,...
2882    [high, visual, acuity, allow, parallel, proces...
Name: abstract_token, Length: 2880, dtype: object

In [66]:
def join_words(raw):
    sentence = ' '.join(raw)
    return sentence

In [27]:
FHP_article.loc[:,'abstract_join'] = FHP_article['abstract_token'].apply(join_words)

/tmp/ipykernel_12512/2594841364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FHP_article.loc[:,'abstract_join'] = FHP_article['abstract_token'].apply(join_words)


In [28]:
FHP_article

,title,artical_date,abstract_list,abstract_clean,abstract_token,abstract_join
0,Influence of forward head posture on muscle ac...,2022 Nov 14.,Go to:\nAbstract\nForward head posture (FHP) i...,is a serious problem causing head and nec...,"[serious, problem, cause, head, neck, disabili...",serious problem cause head neck disability cha...
1,Relative posture between head and finger deter...,2020 Mar 26.,Go to:\nAbstract\nThe hand explores the enviro...,the hand explores the environment for obtaini...,"[hand, explore, environment, obtain, tactile, ...",hand explore environment obtain tactile inform...
2,Head posture impacts mammalian hyoid position ...,"October 16, 2023.",Go to:\nABSTRACT\nInstantaneous head posture (...,instantaneous head posture ihp can extensivel...,"[instantaneous, head, posture, ihp, alter, res...",instantaneous head posture ihp alter rest hyoi...
3,Does head and cervical posture correlate to ma...,2022 Oct 25.,Go to:\nAbstract\nBackground\nThe association ...,background the association of head and cervic...,"[association, head, cervical, posture, maloccl...",association head cervical posture malocclusion...
4,Changes in deep neck muscle length from the ne...,2022 Jan 25.,Go to:\nAbstract\nForward head posture (FHP) i...,is one of the most common postural deviat...,"[common, postural, deviation, deep, neck, musc...",common postural deviation deep neck muscle imb...
...,...,...,...,...,...,...
2878,Characteristics of the postural stability of t...,2023 Jan 4.,Go to:\nAbstract\nObjective: To explore the ch...,to explore the characteristics of lower limb...,"[explore, characteristic, lower, limb, postura...",explore characteristic lower limb postural sta...
2879,Validating an infrared thermal switch as a nov...,2010 Aug 5.,"Go to:\nAbstract\nBackground\nRecently, a nove...",background recently a novel singleswitch acce...,"[background, novel, singleswitch, access, tech...",background novel singleswitch access technolog...
2880,How Not to Be Turned into a Zombie,2018 Oct 31.,Go to:\nAbstract\nThe emerald jewel wasp (Ampu...,the emerald jewel wasp ampulex compressa is r...,"[emerald, jewel, wasp, ampulex, compressa, ren...",emerald jewel wasp ampulex compressa renowned ...
2881,Chronic Cervicogenic Tinnitus Rapidly Resolved...,2016 Mar 23.,Go to:\nAbstract\nIntroduction\nCervicogenic t...,introduction cervicogenic tinnitus is not a g...,"[introduction, cervicogenic, tinnitus, accept,...",introduction cervicogenic tinnitus accept path...


### 유사한 단어 묶기

In [29]:
def word_cleaning(raw):
    raw = raw.replace("chronic neck pain","cnp")
    raw = raw.replace("craniovertebral angle","cva")
    
    return raw

In [30]:
FHP_article.loc[:,'abstract_join'] = FHP_article['abstract_join'].apply(word_cleaning)

In [31]:
len(FHP_article)

2880

In [32]:
FHP_article.isnull().sum()

title             0
artical_date      0
abstract_list     0
abstract_clean    0
abstract_token    0
abstract_join     0
dtype: int64

## 학습데이터, 테스트 데이터 분류

In [33]:
from sklearn.model_selection import train_test_split
FHP_article_train,FHP_article_test = train_test_split(FHP_article,test_size=0.4,random_state=42)

## 키워드 분류하기

### TfidfVectorizer 최적화

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfVectorizer = TfidfVectorizer(max_df=0.60,min_df=0.2,ngram_range=(1,2), stop_words = ['use','method','subject','suggest','have','show','result','include','provide','increase','investigate','significant','measure'])
features_train = tfidfVectorizer.fit_transform(FHP_article_train['abstract_join'])
dictionary_list = tfidfVectorizer.get_feature_names_out()
dictionary_list

array(['activity', 'aim', 'analysis', 'assess', 'body', 'change',
       'compare', 'conclusion', 'control', 'difference', 'different',
       'effect', 'find', 'group', 'head', 'movement', 'muscle', 'patient',
       'perform', 'position', 'postural', 'system', 'test', 'time'],
      dtype=object)

### gridsearch

In [46]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

# LDA 모델을 위한 파라미터 그리드 생성
param_grid = {
    'n_components': [3],
    'learning_decay': [0.7],
    'doc_topic_prior': [1.0],  # 문서-주제 사전확률 (알파)
    'topic_word_prior': [1.0]  # 단어-주제 사전확률 (베타)
}

# LatentDirichletAllocation 모델 인스턴스 생성
lda = LatentDirichletAllocation()

# GridSearchCV 인스턴스 생성
model = GridSearchCV(lda, param_grid, n_jobs=-1, cv=5)

# 데이터는 여기에 당신의 데이터셋을 사용하세요
# X는 문서-단어 행렬이어야 합니다. 예를 들어, CountVectorizer나 TfidfVectorizer를 사용하여 생성할 수 있습니다.
model.fit(features_train)

# 최적 모델
grid_best_model = model.best_estimator_

# 최적의 파라미터와 점수를 출력
print("Best Model's Params: ", model.best_params_)
print("Best Log Likelihood Score: ", model.best_score_)
print("perplexity",grid_best_model.perplexity(features_train) )


Best Model's Params:  {'doc_topic_prior': 1.0, 'learning_decay': 0.7, 'n_components': 3, 'topic_word_prior': 1.0}
Best Log Likelihood Score:  -2445.614601837949
perplexity 30.168143597819803


In [47]:
grid_best_model.fit(features_train)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in grid_best_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index 
    topics_list.append(topics_text)
topics_list_add_grid = [['Topic1', 'Topic2','Topic3'],topics_list]
df_topics_keywords_grid = pd.DataFrame(topics_list_add_grid)
df_topics_keywords_grid=df_topics_keywords_grid.T
df_topics_keywords_grid

,0,1
0,Topic1,muscle system control movement test effect activity postural compare perform
1,Topic2,patient postural body analysis control difference find conclusion aim assess
2,Topic3,head group position change movement time different activity muscle control


In [48]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(grid_best_model,features_train,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

In [49]:
topics_output = grid_best_model.transform(features_train)
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

dominant_topic_number
0    606
1    567
2    555
Name: count, dtype: int64

### test set에 적용해보기

In [50]:
features_test = tfidfVectorizer.transform(FHP_article_test['abstract_join'])
# grid_best_model.fit(features_test)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in grid_best_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
#     print(df_topics[1].values[:10])
#     topics_text = ' '.join(df_topics[1].values[:9])# 시리즈 형식으로 출력 get values from series / index 
#     topics_list.append(topics_text)
# topics_list_add_grid = [['Topic1', 'Topic2','Topic3'],topics_list]
# df_topics_keywords_grid = pd.DataFrame(topics_list_add_grid)
# df_topics_keywords_grid=df_topics_keywords_grid.T
# df_topics_keywords_grid

In [51]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(grid_best_model,features_test,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

In [52]:
topics_output = grid_best_model.transform(features_test)
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

dominant_topic_number
0    401
1    385
2    366
Name: count, dtype: int64

### 전체 데이터에 적용

In [58]:
features = tfidfVectorizer.transform(FHP_article['abstract_join'])

# grid_best_model.fit(features)
import pandas as pd
pd.set_option('display.max_colwidth', None)
## 상위 단어 추출 
## 0 확률 1은 dictionary
topics_list = list()
for topic in grid_best_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
#     # topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index 
#     # topics_list.append(topics_text)
# topics_list_add_grid = [['Topic1', 'Topic2','Topic3'],topics_list]
# df_topics_keywords_grid = pd.DataFrame(topics_list_add_grid)
# df_topics_keywords_grid=df_topics_keywords_grid.T
# df_topics_keywords_grid

In [59]:
df_datas_0 = [grid_best_model.components_[0], dictionary_list]
df_topics_0 = pd.DataFrame(data=df_datas_0).T
df_topics_0 = df_topics_0.dropna()
df_topics_0 = df_topics_0.sort_values(0, ascending=False).reset_index()
df_topics_0.rename(columns = {1 : 'first_components'}, inplace = True)
df_topics_0.rename(columns = {0 : 'score'}, inplace = True)
df_datas_1 = [grid_best_model.components_[1], dictionary_list]
df_topics_1 = pd.DataFrame(data=df_datas_1).T
df_topics_1 = df_topics_1.dropna()
df_topics_1 = df_topics_1.sort_values(0, ascending=False).reset_index()
df_topics_1.rename(columns = {1 : 'second_components'}, inplace = True)
df_topics_1.rename(columns = {0 : 'score'}, inplace = True)
df_datas_2 = [grid_best_model.components_[2], dictionary_list]
df_topics_2 = pd.DataFrame(data=df_datas_2).T
df_topics_2 = df_topics_2.dropna()
df_topics_2 = df_topics_2.sort_values(0, ascending=False).reset_index()
df_topics_2.rename(columns = {1 : 'third_components'}, inplace = True)
df_topics_2.rename(columns = {0 : 'score'}, inplace = True)
df_topic = pd.concat([df_topics_0.loc[:20,['score','first_components']],df_topics_1.loc[:20,['score','second_components']],df_topics_2.loc[:20,['score','third_components']]],axis=1)
df_topic

,score,first_components,score,second_components,score,third_components
0,171.028414,muscle,151.49683,patient,187.34375,head
1,98.10041,system,138.011183,postural,131.25437,group
2,89.704644,control,136.646549,body,106.321877,position
3,89.506852,movement,93.92038,analysis,102.302061,change
4,83.888304,test,72.091307,control,80.372577,movement
5,76.245857,effect,59.796176,difference,74.795497,time
6,64.985638,activity,49.949548,find,60.520678,different
7,62.819511,postural,47.213351,conclusion,52.725529,activity
8,60.539692,compare,46.192095,aim,34.957128,muscle
9,53.93291,perform,45.579726,assess,33.202424,control


In [60]:
import pyLDAvis
import pyLDAvis.lda_model
vis_grid = pyLDAvis.lda_model.prepare(grid_best_model,features,tfidfVectorizer)
pyLDAvis.enable_notebook()
components_grid_display = pyLDAvis.display(vis_grid)
components_grid_display

In [61]:
df_topics_score = pd.DataFrame(topics_output)
df_topics_score['dominant_topic_number']=np.argmax(topics_output, axis=1)
df_topics_score['dominant_topic_number'].value_counts()

dominant_topic_number
0    401
1    385
2    366
Name: count, dtype: int64

In [62]:
df_topics_score['dominant_topic_number']

0       2
1       2
2       0
3       0
4       1
       ..
1147    2
1148    1
1149    1
1150    1
1151    2
Name: dominant_topic_number, Length: 1152, dtype: int64

## 토픽 분석하기

### topic별로 분류하기

In [ ]:
FHP_article = FHP_article.reset_index()
FHP_article.loc[:,'article_topic'] = df_topics_score['dominant_topic_number']

In [ ]:
FHP_article.isnull().sum()

### topic 별 많이 등장하는 단어 추출

In [ ]:
FHP_article.query("article_topic == 0")

In [ ]:
len(FHP_article.query("article_topic == 1"))

In [ ]:
FHP_article.query("article_topic == 2")

In [ ]:
FHP_article.to_csv('FHP_article.csv',index=False,encoding='utf-8')

## 결론
- 논문
- 중복 논문을 삭제하고, 결측치를 제거하니 데이터가 1000개정도밖에 남지 않음
- 다른 사이트로 다시 스크래핑 후 재시도

### Count vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words=[], ngram_range=(1, 2))

In [ ]:
FHP_article.query("article_topic == 2")['abstract_join']

In [ ]:
count_vect.fit(FHP_article['abstract_join'])

In [ ]:
count_vect.vocabulary_

In [ ]:
dtm_topic_0 = count_vect.transform(FHP_article.query("article_topic == 0")['abstract_join'])
dtm_array_0 = dtm_topic_0.toarray()
df_dtm_0 = pd.DataFrame(dtm_array_0, columns = vocabs)
df_first_topic = df_dtm_0.sum().sort_values(ascending=False)[:20]
df_first_topic = df_first_topic.reset_index()
df_first_topic.rename(columns = {'index' : 'first'}, inplace = True)
df_first_topic.rename(columns = {0 : 'count'}, inplace = True)

In [ ]:
dtm_topic_1 = count_vect.transform(FHP_article.query("article_topic == 1")['abstract_join'])
dtm_array_1 = dtm_topic_1.toarray()
df_dtm_1 = pd.DataFrame(dtm_array_1, columns = vocabs)
df_second_topic = df_dtm_1.sum().sort_values(ascending=False)[:20]
df_second_topic = df_second_topic.reset_index()
df_second_topic.rename(columns = {'index' : 'second'}, inplace = True)
df_second_topic.rename(columns = {0 : 'count'}, inplace = True)

In [ ]:
dtm_topic_2 = count_vect.transform(FHP_article.query("article_topic == 2")['abstract_join'])
dtm_array_2 = dtm_topic_2.toarray()
df_dtm_2 = pd.DataFrame(dtm_array_2, columns = vocabs)
df_third_topic = df_dtm_2.sum().sort_values(ascending=False)[:20]
df_third_topic = df_third_topic.reset_index()
df_third_topic.rename(columns = {'index' : 'third'}, inplace = True)
df_third_topic.rename(columns = {0 : 'count'}, inplace = True)

In [ ]:
vocabs = count_vect.get_feature_names_out()
vocabs[:2]
vocabs.shape

In [ ]:
df_topic = pd.concat([df_first_topic,df_second_topic,df_third_topic],axis=1)

In [ ]:
df_topic